In [1]:
import pandas as pd

df = pd.read_csv("../../data/sample_reviews.csv")
df.head()

C:\Users\hoyong\AppData\Local\Temp\ipykernel_25264\4112985669.py:3: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data/sample_reviews.csv")


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [2]:
import re
from nltk.corpus import stopwords


def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return " ".join(words)


df['cleaned_review'] = df['reviews.text'].apply(clean_text)

In [3]:
from textblob import TextBlob


def get_sentiment(text):
    return TextBlob(text).sentiment.polarity


df['sentiment_score'] = df['cleaned_review'].apply(get_sentiment)

df['sentiment'] = df['sentiment_score'].apply(
    lambda x: 'positive' if x > 0.2 else 'negative' if x < -0.2 else 'neutral'
)

In [4]:
import os

from dotenv import load_dotenv
import httpx
import openai

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

http_client=httpx.Client(verify=False)
client = openai.OpenAI(
    api_key=api_key,
    http_client=http_client
)


def generate_summary(reviews):
    prompt = "Summarize the following customer feedback:\n" + "\n".join(reviews)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=200
    )
    return response.choices[0].message.content


summary = generate_summary(df['cleaned_review'].sample(10).tolist())
print(summary)

Customers generally have positive feedback about the Fire tablet. They appreciate its value for money, good picture quality for videos, and its ease of synchronization with older tablets. It's also praised for its functionality in reading and saving media, watching movies, and playing games. Some customers bought it for their children and found it kept them occupied. One customer liked the tablet's ability to perform various tasks such as checking credit card balance, ordering from Amazon, and listening to the radio. However, a few customers noted some negatives such as software issues, incompatibility with Chromecast, and the need to push the start button several times to unlock the screen. Overall, the Fire tablet is considered a decent buy.


In [5]:
# Save as app.py
import streamlit as st

st.title("Customer Review Dashboard")
st.write(summary)
st.bar_chart(df['sentiment'].value_counts())

2025-08-03 14:02:42.957 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 14:02:43.516 
  command:

    streamlit run c:\Users\hoyong\Desktop\smart_feedback_analyzer\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-03 14:02:43.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 14:02:43.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 14:02:43.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 14:02:43.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 14:02:43.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-03 14:02:45

DeltaGenerator()